In [13]:
import dbt
import json
import IPython.display as ipd
import utilities 
import Segment
# import librosa

In [42]:
song1_name = "nobody_s_perfect.wav"
# segmentino_name = song1_name.replace(".wav", "") + "_segmentino.txt"
# song1_data, sr = dbt.load_and_resample(song1_name)
# def create_segment_audio(data, name, segment_name, sr):
#     segments = Segment.extract_segment(segment_name)
#     start = int(segments[idx].start * sr)
#     end = int(segments[idx].end * sr)
#     utilities.slice_song_and_save(data, start, end, sr, name, str(idx))

# create_segment_audio(song1_data, song1_name, segmentino_name, sr)

In [5]:
print(len(song1_data) / sr)

237.68816326530612


In [ ]:
import Segment
import Alignment
json_song1_name = song1_name.replace(".wav", "") + ".json"
with open(json_song1_name, "w") as json_file:
    dump_data = []
#     song1_segments = Segment.extract_segment(segmentino_name)
    song_data, sr  = dbt.load_and_resample(song1_name)
    song1_segments = [Segment.Segment(0, len(song_data) / sr, "")]
    for idx in range(len(song1_segments)):
#         song_data,sr = dbt.load_and_resample(song1_name.replace(".wav", "") + "_" + str(idx) + ".wav")
        print("Loaded song_data with sr", sr, "length", len(song_data) / sr)
        print("Segment info:", song1_segments[idx].toString(), "idx", idx)    
        #Get Downbeats
        downbeats = dbt.extract_downbeat(4,song_data)
        
        time_1, beat_num1 = downbeats[:,0].tolist(), downbeats[:,1].tolist()
#         if time_1[0] > 0.07:
#             time_1 = [0.0] + time_1
#             beat_num1 = [(beat_num1[0] - 1) % 4] + beat_num1
        
        print("length time_1", len(time_1))
        #Get Chord
        chord_dict, chord_result1 = dbt.get_chords_old(time_1, song_data, sr)
        #Smoothen the chords
        print("Chord_result",chord_result1)
        new_chord_result, isAligned = Alignment.chord_alignment(chord_result1)
        song1_segments[idx].useable = isAligned
        
        #Squash the same beats together
        if isAligned:
            boolean_array = [False for i in range(len(new_chord_result))]
            for i in range(len(new_chord_result)):
                if i == 0 or new_chord_result[i] != new_chord_result[i-1]:
                    boolean_array[i] = True
            new_chords = []
            new_time = []
            for i in range(len(boolean_array)):
                if boolean_array[i]:
                    new_chords.append(new_chord_result[i])
                    new_time.append(time_1[i])
            new_chord_result = new_chords
            time_1 = new_time
            
        #Average time per beat
        avg_time_beat = 0
        for i in range(len(time_1) - 1):
            avg_time_beat += time_1[i+1] - time_1[i]
        avg_time_beat = avg_time_beat / len(time_1)
        print("Average time_per_chord", avg_time_beat)
        
        if not isAligned:
            print("Cannot align chord")
        
        song1_segments[idx].chords = new_chord_result
        song1_segments[idx].downbeats = time_1
        song1_segments[idx].average_time_per_beat = avg_time_beat
        
        
        beat_num1 = [1 for i in range(len(time_1))]
        
        dbt.print_and_save_chord(song1_name, time_1, song1_segments[idx].chords, str(idx) )
        dbt.print_and_save_downbeat(song1_name, time_1, beat_num1, str(idx))
        
        dump_data.append(song1_segments[idx].getJSON())    
    json.dump(dump_data, json_file, indent=4)
        

Loaded song_data with sr 44100 length 221.4661224489796
Segment info: Start:0 End:221.4661224489796 Duration: 221.4661224489796 Type: downbeat_length:0 chord_length:0 idx 0


In [27]:
sample,sr = dbt.load_and_resample(song1_name.replace(".wav", "") + "_" + str(1) + ".wav")
downbeats = dbt.extract_downbeat(4,sample)
dbt.print_and_save_downbeat(song1_name, downbeats[:,0], downbeats[:,1])
# time_1, beat_num1 = dbt.filter_downbeat(downbeats)
time_1 = downbeats[:,0]
chord_dict, chord_result = dbt.get_chords_old(time_1, sample, sr)
dbt.print_and_save_chord(song1_name, downbeats[:,0], chord_result)



Saving downbeat of replay.wav
index 0 : 0.02 : 0.35 duration: 0.32999999999999996 [(0., 0.4, 'N')]
index 1 : 0.35 : 0.68 duration: 0.33000000000000007 [(0., 0.4, 'F#:min')]
index 2 : 0.68 : 1.01 duration: 0.32999999999999996 [(0., 0.4, 'N')]
index 3 : 1.01 : 1.34 duration: 0.33000000000000007 [(0., 0.4, 'N')]
index 4 : 1.34 : 1.66 duration: 0.31999999999999984 [(0., 0.4, 'D:maj')]
index 5 : 1.66 : 2.0 duration: 0.3400000000000001 [(0., 0.4, 'D:maj')]
index 6 : 2.0 : 2.33 duration: 0.33000000000000007 [(0., 0.4, 'N')]
index 7 : 2.33 : 2.66 duration: 0.33000000000000007 [(0., 0.4, 'D:maj')]
index 8 : 2.66 : 2.98 duration: 0.31999999999999984 [(0., 0.4, 'N')]
index 9 : 2.98 : 3.31 duration: 0.33000000000000007 [(0., 0.4, 'A:maj')]
index 10 : 3.31 : 3.65 duration: 0.33999999999999986 [(0., 0.4, 'N')]
index 11 : 3.65 : 3.97 duration: 0.3200000000000003 [(0., 0.4, 'A:maj')]
index 12 : 3.97 : 4.3 duration: 0.3299999999999996 [(0., 0.4, 'N')]
index 13 : 4.3 : 4.63 duration: 0.33000000000000007

index 114 : 37.6 : 37.93 duration: 0.3299999999999983 [(0., 0.4, 'N')]
index 115 : 37.93 : 38.27 duration: 0.3400000000000034 [(0., 0.4, 'N')]
index 116 : 38.27 : 38.59 duration: 0.3200000000000003 [(0., 0.4, 'N')]
index 117 : 38.59 : 38.92 duration: 0.3299999999999983 [(0., 0.4, 'N')]
index 118 : 38.92 : 39.25 duration: 0.3299999999999983 [(0., 0.4, 'N')]
index 119 : 39.25 : 39.58 duration: 0.3299999999999983 [(0., 0.4, 'N')]
index 120 : 39.58 : 39.91 duration: 0.3299999999999983 [(0., 0.4, 'N')]
index 121 : 39.91 : 40.24 duration: 0.3300000000000054 [(0., 0.4, 'N')]
index 122 : 40.24 : 40.57 duration: 0.3299999999999983 [(0., 0.4, 'N')]
index 123 : 40.57 : 40.9 duration: 0.3299999999999983 [(0., 0.4, 'N')]
index 124 : 40.9 : 41.23 duration: 0.3299999999999983 [(0., 0.4, 'N')]
index 125 : 41.23 : 41.56 duration: 0.3300000000000054 [(0., 0.4, 'N')]
index 126 : 41.56 : 41.89 duration: 0.3299999999999983 [(0., 0.4, 'N')]
index 127 : 41.89 : 42.22 duration: 0.3299999999999983 [(0., 0.4, '